In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25903744
paper_name = 'trikka_makris_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits.txt')

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 100 x 1


In [10]:
original_data.head()

,ORF
0,YGR2014w
1,YIL087C
2,YJL046W
3,YGL021W
4,YNL242W


In [15]:
original_data['orf'] = original_data['ORF'].astype(str)

In [16]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [17]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [18]:
original_data.loc[original_data['orf']=='YGR2014W','orf'] = 'YGR204W'

In [19]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, orf]
Index: []


In [20]:
original_data['data'] = 1

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(100, 1)

# Load & process tested strains

In [27]:
tested = pd.read_excel('raw_data/Res Gen Strain Collection.xls', sheet_name='Sheet1', skiprows=1)

In [28]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,RG,notes,QC,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,405,YAL002W,BY4741,01_1,1.0,E,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,402,YAL004W,BY4741,01_1,1.0,E,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,403,YAL005C,BY4741,01_1,1.0,E,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,401,YAL007C,BY4741,01_1,1.0,D,12.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,400,YAL008W,BY4741,01_1,1.0,D,11.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
tested['orf'] = tested['ORF name'].astype(str)

In [30]:
tested['orf'] = clean_orf(tested['orf'])

In [31]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [32]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [33]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

                                                    record no. ORF name  \
index_input                                                               
4705                                                       NaN      NaN   
4706         2406    YOR150W BY4741  15_2    70      A     ...      NaN   
4707         2414    YOR158W BY4741  15_2    70      A     ...      NaN   
4708         5246    YLR337C BY4741  12_5    70      A     ...      NaN   
4709         5247    YLR338W BY4741  12_5    70      A     ...      NaN   
...                                                        ...      ...   
4884         3257    YBR118W BY4741  2_3                   ...      NaN   
4885         803     YMR217W BY4741  13_4                  ...      NaN   
4886         1426    YIL033C BY4741  00_17b                ...      NaN   
4887         336     YER014W BY4741  05_2                  ...      NaN   
4888         3224    YBR086C BY4741  2_3                   ...      NaN   

            strain batch

In [36]:
# Delete strains on plates >= 70
tested = tested.loc[t,:]

In [37]:
tested_orfs = tested['orf'].unique()

In [38]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YDR361C', 'YKL096C-B', 'YOR029C']

In [39]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [41]:
original_data.shape

(4626, 1)

In [46]:
original_data.sum()

data    97
dtype: int64

# Prepare the final dataset

In [42]:
data = original_data.copy()

In [43]:
dataset_ids = [22258]
datasets = datasets.reindex(index=dataset_ids)

In [44]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [45]:
data.head()

dataset_id,22258
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [47]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [48]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22258
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [49]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [50]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [51]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22258       
data_type       value valuez
gene_id orf                 
2       YAL002W     0    0.0
1863    YAL004W     0    0.0
4       YAL005C     0    0.0
5       YAL007C     0    0.0
6       YAL008W     0    0.0

# Print out

In [52]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')